In [1]:
# https://www.youtube.com/watch?v=vWO0oglqTAQ

!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 28.4MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 18.0MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

#metryka sukcesu
from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/DW-Matrix/samochody2020/ML-car-2020-01/"

/content/drive/My Drive/Colab Notebooks/DW-Matrix/samochody2020/ML-car-2020-01


In [4]:
df=pd.read_hdf('./data/car.h5')
df.shape

(106494, 155)

In [0]:
#przygotowanie cech - faktoryzacja cech - np id dla colorów
SUFFix_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue
  
  factorized_values = df[feat].factorize()[0]
  if SUFFix_CAT in feat:
    df[feat] = factorized_values
  else:  

    df[feat + SUFFix_CAT] =  factorized_values

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str(x).split('cm')[0].replace(' ', '')))


In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error') # otrzymamy wartosc ujemna
  return np.mean(scores), np.std(scores)

In [8]:
feats =['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa',
'seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat',
'feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

## XGBoost 
xgb_params = {'max_depth': 5,
              'n_estimators': 50,
              'leraning_rate': 0.1,
              'seed':0
              }

run_model(xgb.XGBRegressor(**xgb_params), feats)

[12:20:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:21:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:21:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [34]:
#hyperopt - probkuje w którą strone zmierzac aby było dobrze  wynik to optymalne zmienna + najlepszy wynik

def obj_func(params):
    print("Training with params: ")
    print(params)
    mean_mea, score_std = run_model(xgb.XGBRegressor(**params),feats)
    return {'loss': np.abs(mean_mea), 'status': STATUS_OK}

#space - granice dla optymalizatora rozsadne ramy 
xgb_reg_params = {
     'learning_arate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)), #bylo 0.05
     'max_death': hp.choice('max_death', np.arange(5, 16, 1, dtype = int)), #glebokosc poszczegolnych drzew 
     'subsample': hp.quniform('subsample', 0.5, 1, 0.03), #wartosci od 0.5 do 1 i krok 0.05
     'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05), 
     'objective': 'reg:squarederror', #narzekanie bledy patrz wyzej
     'n_estimators': 100,
     'seed': 0,
}   

## run 
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=12) 

best 

Training with params: 
{'colsample_bytree': 0.75, 'learning_arate': 0.05, 'max_death': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8099999999999999}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_arate': 0.15000000000000002, 'max_death': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.72}
Training with params: 
{'colsample_bytree': 0.75, 'learning_arate': 0.15000000000000002, 'max_death': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.72}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_arate': 0.2, 'max_death': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_arate': 0.2, 'max_death': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9299999999999999}
Training with params: 
{'colsa

{'colsample_bytree': 0.9500000000000001,
 'learning_rate': 4,
 'max_death': 9,
 'subsample': 0.78}

In [46]:
xgb_params = {'max_depth': 9,
              'n_estimators': 50,
              'leraning_rate': 0.3,
              'seed':0,
              'objective': 'reg:squarederror',
              }

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-8124.65189710521, 39.31823480417909)

In [47]:
xgb_params = {'max_depth': 10,
              'n_estimators': 50,
              'leraning_rate': 0,
              'seed':0,
              'objective': 'reg:squarederror',
              }

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-7980.848703913878, 71.02226534839897)

In [9]:
xgb_params = {'max_depth': 10,
              'n_estimators': 60,
              'leraning_rate': 0,
              'seed':0,
              'objective': 'reg:squarederror',
              }

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-7877.797004035154, 72.54231146059084)

In [11]:
xgb_params = {'max_depth': 10,
              'n_estimators': 66,
              'leraning_rate': 4,
              'seed':0,
              'objective': 'reg:squarederror',
              }

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-7826.381632427695, 75.2947480979551)

In [12]:
xgb_params = {'max_depth': 12,
              'n_estimators': 66,
              'leraning_rate': 4,
              'seed':0,
              'objective': 'reg:squarederror',
              }

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-7731.701757614345, 81.56266338812601)